In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import psycopg2

In [ ]:
# Set your Spotify API credentials
client_id = "your_id"
client_secret = "your_secret"

In [ ]:
# Initialize the Spotipy client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

In [ ]:
# Define seed genres
seed_genres = ["classical", "country", "chill", "edm", "folk", "gospel", "hip-hop", "jazz", "k-pop", "latin", "metal", "pop", "r-n-b", "rock", "minimal-techno"]

In [ ]:
# Initialize an empty list to store track data
all_track_data = []

In [ ]:
# Get recommended tracks for each seed genre
for seed_genre in seed_genres:
    recommended_tracks = sp.recommendations(seed_genres=[seed_genre], limit=3)
    for track in recommended_tracks["tracks"]:
        track_data = {
            "track_name": track["name"],
            "artist": ", ".join(artist["name"] for artist in track["artists"]),
            "album": track["album"]["name"],
            "genre": seed_genre,
            "preview_url": track["preview_url"]
        }
        all_track_data.append(track_data)

In [ ]:
# Create a DataFrame from the track data
df = pd.DataFrame(all_track_data)

In [ ]:
# Print the DataFrame
print(df)

In [ ]:
# # Connect to PostgreSQL database
db_params = {
    "host": "localhost",
    "dbname": "xx",  # Replace with your desired database name
    "user": "xx",  # Replace with your PostgreSQL username
    "password": "xx",  # Replace with your PostgreSQL password
    "port": "xx" # Replace with your PostgreSQL port
}

In [ ]:
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

In [ ]:
# Create a table (if not exists)
cur.execute("""
    CREATE TABLE IF NOT EXISTS recommended_tracks (
        track_name VARCHAR,
        artist VARCHAR,
        album VARCHAR,
        genre VARCHAR,
        preview_url VARCHAR
    )
""")
conn.commit()

In [ ]:
# # Insert data into the table
for _, row in df.iterrows():
    cur.execute("""
        INSERT INTO recommended_tracks (track_name, artist, album, genre, preview_url)
        VALUES (%s, %s, %s, %s, %s)
    """, (row["track_name"], row["artist"], row["album"], row["genre"], row["preview_url"]))

conn.commit()
conn.close()